## Описание

Классификация результатов разделения по сюжетам. 

## Зависимости

In [8]:
# %load ./src/imports/basic_imports.py
import os
import pandas as pd
import numpy as np


In [9]:
# %load ./src/imports/gdown_imports.py

! pip install gdown
import gdown


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# %load ./src/imports/random_imports.py

from random import sample, randint


In [10]:
# зависимости из классификатора
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
_stopwords = stopwords.words('russian')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eu2233-6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Функции обработки текстовых данных

In [11]:
# функция из классификатора - векторизация
def vectorize(data, _type, sw=None):
    
    if _type == 'count':
        vectorizer = CountVectorizer(stop_words=sw)
        X = vectorizer.fit_transform(data['text'])
    if _type == 'tfidf':
        vectorizer = TfidfVectorizer(stop_words=sw)
        X = vectorizer.fit_transform(data['text'])
        
    return X, vectorizer.get_feature_names_out()

In [12]:
# функция из классификатора - очистка базы
def clean(data, cols_drop, pattern, replacement, lemmatize=True):
    
    if cols_drop != None:
        data = data.drop(columns=cols_drop)
    data = data.fillna('')
    data['text'] = data['name'] + " " + data['author'] + " " + data['technologies']
    data['text'] = data['text'].astype('str')
    data['text'] = data['text'].apply(lambda x: x.lower())
    data['text'] = data.text.str.replace(pattern, replacement)
    if lemmatize:
        m = Mystem()
        data['text'] = data.text.apply(lambda x: ''.join(m.lemmatize(x)).rstrip())
    
    return data

## Параметры

In [13]:
# определяем колонку по которой будем связывать все таблицы
global_key = 'regNumber'

# ссылка на выгрузку данных ГК
url_text_objects = './out/gk_out/df_merge_labelled_enriched - df_merge_labelled_enriched.csv'

#### Изображения
- DBSCAN чистая выборка с лучшими параметрами (eps = 0.387,sample = 2) 
- KMeans чистая выборка с лучшими значениями метрик на валидационной выборке (k = 93) 

In [14]:
# ссылки на результаты кластаризации
url_dict = {
    'DBSCAN_clear':'./out/classter_img_out/DBSCAN_clear_1346.csv',
    'KMeans_clear_best':'./out/classter_img_out/KMeans_full_best_1406.csv',
    'df_text_classter':'./out/classter_text_out/'
    }

## Вспомогательные функции

In [15]:
# %load ./src/functions/csv2DF.py

# загрузка файлов по словарю
# name - имя
# link_dir - словарь
# sep = "," - разделитель
def csv2DF(name, link_dir = url_dict, sep = ","):
    url = link_dir[name]
    df = pd.read_csv(url)
    return df


In [16]:
# %load ./src/functions/save2csv.py

# сохраняем в файл результаты
def save2csv(df, param='data'):
    file_name = './out/' + param + '_' + str(randint(100, 2000)) +'.csv'
    df.to_csv(file_name, index= False )


In [17]:
# %load ./src/functions/csv2DF.py

# загрузка файлов по словарю
# name - имя
# link_dir - словарь
# sep = "," - разделитель
def csv2DF(name, link_dir = url_dict, sep = ","):
    url = link_dir[name]
    df = pd.read_csv(url, encoding = 'utf8', sep = sep )
    return df


## Загрузка результатов выгрузки данных из ГК

In [18]:
objects = pd.read_csv(url_text_objects, encoding = 'utf8')

In [19]:
objects.head()

,id,name,author,regNumber,invNumber,gikNumber,type,statusId,nativeId,date_created,typology,description,technologies,image,connectedness
0,5c3e0d8893fa687ca4f7f2cc,"Эскиз декорации. Сцена с фигурой. ""В списках н...","Макушенко Владимир Александрович ,Твардовская ...",3613323,Ж 1308,КП 323343,0,6,1186967,Пост. 1975 г.,живопись,В чёрной раме на чёрном фоне в центре серые пя...,"Фанера, темпера, мелки, ткань, аппликация.",http://goskatalog.ru/muzfo-imaginator/rest/ima...,0.0
1,5c3e0ff893fa687ca41a64ad,Екатерининский дворец,Махаев Н. М.,3640287,ЕД-865-X,ЦС КП-19894,0,6,475975,1827,живопись,Изображен Екатерининский дворец со стороны сев...,"масло, холст",http://goskatalog.ru/muzfo-imaginator/rest/ima...,0.0
2,5c3e190393fa687ca49e7935,Портрет. И. И. Махаев в роли. Неустановленный ...,[Ларин Юрий Николаевич],9696309,Ж 1548,ГЦТМ КП 331864,0,6,9817591,1985 г. 1985,живопись,Изображен в повороте корпусом вправо молодой т...,"холст, масло",http://goskatalog.ru/muzfo-imaginator/rest/ima...,0.0
3,5c3e190493fa687ca49e7fb8,Портрет. Махаев И. И.,Гусев Владимир Сергеевич,9696756,Ж 1549,ГЦТМ КП 331865,0,6,9817991,1980-е гг. 1980-1989,живопись,На фоне стены изображен поколенно корпусом в ф...,"Холст, масло",http://goskatalog.ru/muzfo-imaginator/rest/ima...,0.0
4,5c3e196393fa687ca4a40b12,Портрет. И. И. Махаев,Лукьянов Виктор Евгеньевич,9858879,NaN,ГЦТМ КП 332002,0,6,9976566,2008 г. 2008,живопись,На сиреневом фоне изображен корпусом в профиль...,"холст, масло",http://goskatalog.ru/muzfo-imaginator/rest/ima...,0.0


In [20]:
objects_cleaned = clean(data=objects, cols_drop=['id', 'invNumber', 'gikNumber', 'type', 'nativeId',
                                                 'description', 'image'],
                        pattern='[\d+\\.\\,\\!\\?\\-\"\'\\:\\;\\`\\)\\(\\{\\}\\[\\]x+v+i+\\«\\»\\—\\/\\\\\n]', replacement='')

/tmp/ipykernel_678129/4254471794.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data.text.str.replace(pattern, replacement)


## Загрузка результатов класстаризации изображений

In [23]:
df_DBSCAN_clear = csv2DF('DBSCAN_clear')
df_KMeans_clear_best= csv2DF('KMeans_clear_best')

#df_text_classter = csv2DF('df_text_classter')

## Классификация

####  Отдельно LR

In [24]:
X_text, features_text = vectorize(data=objects_cleaned, _type='count', sw=_stopwords)

In [25]:
# векторизируем текст
# X_text, features_text = vectorize(data=objects_cleaned, _type='count', sw=_stopwords)

# параметры разделения на тест и трейн взяты из классsификатора
X_train, X_test, y_train, y_test = train_test_split(
    X_text, objects_cleaned['connectedness'], test_size=0.2,
    stratify=objects_cleaned['connectedness'], random_state=123456)

# параметры регрессии взяты из классификатора
lr_object = LogisticRegression(multi_class='multinomial', max_iter=5000)
lr_object.fit(X_train, y_train)

print(classification_report(y_test, lr_object.predict(X_test)))
print(classification_report(y_train, lr_object.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98        68
         1.0       0.76      0.86      0.81        56
         2.0       0.90      0.88      0.89       155
         3.0       0.74      0.65      0.69        31
         4.0       1.00      0.96      0.98        25
         5.0       1.00      1.00      1.00        10

    accuracy                           0.89       345
   macro avg       0.90      0.89      0.89       345
weighted avg       0.89      0.89      0.89       345

              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00       273
         1.0       0.98      0.94      0.96       222
         2.0       0.96      0.98      0.97       619
         3.0       0.93      0.93      0.93       121
         4.0       1.00      0.99      1.00       102
         5.0       1.00      1.00      1.00        42

    accuracy                           0.97      1379
   macro avg       0.98

#### Функции для работы с предсказаниями по изображениям

In [26]:
# применяем LR для переданной выборки

# df_for_class - выборка из датафрейма изображений для которых получаем предсказание, 
# df_objects = objects_cleaned, - выборка с текстами
# lr = lr_object - натренерованная lr
# X = X_text - результат работы функции vectoraze - разреженная матрица текстов
# class_true_col = 'connectedness' - id колонки с реальным значением 

# Возвращает predict функции и список с истенными значениями ['connectedness'] для выбранных элементов

def get_LR_predict(df_for_class, 
                   df_objects = objects_cleaned, 
                   X = X_text, 
                   class_true_col = 'connectedness'):
     key_list = df_for_class[global_key].tolist()        
          
     test_index_list = df_objects.index[df_objects[global_key].isin(key_list)].tolist()
     train_index_list = df_objects.index[~df_objects[global_key].isin(key_list)].tolist()
     
     # вот так мы можем получить из результата работы функции vectoraze нужные нам строки из разреженной матрицы. No comments.
     #X_sel = np.array(pd.DataFrame(X.toarray()).iloc[index_list])        
     X_test = np.float32(X.toarray())[test_index_list]         
     X_train = np.float32(X.toarray())[train_index_list] 
    
     y_test = df_objects.loc[test_index_list, class_true_col]
     y_train = df_objects.loc[train_index_list, class_true_col]
    
     # Кажется логичней тренеровать каждый раз на чистой выборке для лучшего результата
     lr = LogisticRegression(multi_class='multinomial', max_iter=5000)   
     
     # параметры разделения на тест и трейн взяты из классsификатора
     #X_train, X_test, y_train, y_test = train_test_split(
     #X_train_index, df_objects_train['connectedness'], test_size=0.2,
     #stratify=df_objects_train['connectedness'], random_state=123456)

     # параметры регрессии взяты из классификатора     
     lr.fit(X_train, y_train)
     
     return lr.predict(X_test)

In [27]:
# формируем фрейм для классификации
# df - датафрейм изображений,
# labels - метки класстера полученные, 
# claster_num - номер нужного кластера
def get_Classter_LR_predict(df, labels, claster_num,
                                 col_plot = 'labels'):
    df_temp = pd.DataFrame({global_key: df[global_key], 'img': df['img'], 'title': df['title'], 'labels':labels})
    rezult_df = df_temp.loc[df_temp[col_plot] == claster_num].copy()
    
    predict_label, true_label = get_LR_predict(rezult_df)
    rezult_df['predict'] = list(predict_label)
    rezult_df['true'] = list(true_label)

    return rezult_df, predict_label, true_label

In [28]:
# формируем фрейм для классификации по всем кластерам
# df - датафрейм с результатами разделения на сюжеты на основе изображений,
# df_objects - результатов выгрузки данных из ГК
# col_plot = 'labels' - id колонки с номером сюжета
# class_predict_col = 'class_predict' - id колонки с предсказанием
# class_true_col = 'connectedness' - id колонки с реальным значением 

def get_Classter_LR_predict_full(df, 
                                 df_objects = objects_cleaned,
                                 col_plot = 'labels', 
                                 class_predict_col = 'class_predict', 
                                 class_true_col = 'connectedness'): 
    #full_df = pd.DataFrame(columns=[global_key, 'img', 'title', 'labels', 'class_predict', 'class_true'])    
    full_df = pd.DataFrame()
    df = df.drop(columns=['img', 'title'])
    labels = df[col_plot].tolist()
    
    for claster_num in list(np.unique(labels)):
        rezult_df = df.loc[df[col_plot] == claster_num].copy()
        
        predict_label = get_LR_predict(rezult_df)
        rezult_df[class_predict_col] = list(predict_label)        
        
        full_df = pd.concat([full_df, rezult_df])
        
    key_list = full_df[global_key].tolist()       
    df_objects_temp = df_objects.loc[df_objects[global_key].isin(key_list)].copy()
    df_objects_temp = df_objects_temp[[global_key, 'name', class_true_col]]
    full_df = full_df.merge(df_objects_temp, how='left', on=global_key)
    return full_df

#### Функции оценки результатов предсказаний

In [29]:
# получаем результаты из конкретного кластера таблицы

# df таблица с результатами классификации
# col_plot = 'labels' - id колонки с номером сюжета
# class_predict_col = 'class_predict' - id колонки с предсказанием
# class_true_col = 'connectedness' - id колонки с реальным значением

def get_claster_LR_rezult(df, claster_num, 
                          col_plot = 'labels', 
                          class_predict_col = 'class_predict', 
                          class_true_col = 'connectedness'):    
    class_df = df.loc[df[col_plot] == claster_num].copy()
    class_predict = class_df[class_predict_col].tolist()
    class_true = class_df[class_true_col].tolist()
    
    return class_predict, class_true

In [30]:
# получаем таблицу-основу для построения гистораммы с результатами оценки 'accuracy' по разным кластерам
# возвращает датафрейм вида:
#  - 'accuracy' - интервал точности
#  - 'count_labels' - количество сюжетов, оценки точности которых попали в интервал
#  - 'labels' - список сюжетов, оценки точности которых попали в интервал
#  - 'median_len'- медианное значение количества элементов внутри список сюжетов, оценки точности, которых попали в интервал

# df таблица с результатами оценки точности вида
#  - 'accuracy' - точность
#  - 'labels' - номер сюжета
#  - 'claster_len' - количества элементов внутри список сюжетов
from statistics import median

def get_beans_df(df):
    list_rezult = []
    for i in range(0,10,2):
      min_score, max_score = i/10, 0.2+i/10          
      min_score_title = str(min_score) if len(str(min_score)) <=3 else str(min_score)[0:4]
      max_score_title = str(max_score) if len(str(max_score)) <=3 else str(max_score)[0:4]        
      
      if max_score == 1:
         temp_df = df[(df['accuracy'] >= min_score) & (df['accuracy']<=max_score)].copy()
      else:
         temp_df = df[(df['accuracy'] >= min_score) & (df['accuracy']<max_score)].copy()
      labels_list = [str(t) for t in temp_df['labels'].tolist()]
      median_len =  median(temp_df['claster_len'].tolist())
        
      temp_dir = {'accuracy':min_score_title + '-' +  max_score_title, 'count_labels':len(temp_df), 'labels':','.join(labels_list), 'median_len':median_len}
      list_rezult.append(temp_dir)
    return pd.DataFrame(list_rezult)

In [31]:
# выводим accuracy по всем классам
# возвращает датафрейм вида:
#  - 'accuracy' - точность
#  - 'labels' - номер сюжета
#  - 'claster_len' - количества элементов внутри список сюжетов

# df таблица с результатами классификации
# col_plot = 'labels' - id колонки с номером сюжета
# class_predict_col = 'class_predict' - id колонки с предсказанием
# class_true_col = 'connectedness' - id колонки с реальным значением
def score_DF(df,
                          col_plot = 'labels', 
                          class_predict_col = 'class_predict', 
                          class_true_col = 'connectedness'): 
    result_list = []
    labels_list = np.unique(df[col_plot].tolist())
    
    for claster_num in labels_list:
         temp_dict = {'labels':claster_num, 'claster_len':len(df[df[col_plot]==claster_num])}
         y_pred, y_true = get_claster_LR_rezult(df, claster_num, col_plot = col_plot, class_predict_col = class_predict_col, class_true_col =class_true_col)
         
         temp_dict.update({'accuracy':accuracy_score(y_true, y_pred)})
            
         result_list.append(temp_dict)
    return pd.DataFrame(result_list)

In [32]:
# проводим оценку и выводим результаты
# печатает общее количество сюжетов и количество сюжетов для которых оценка ниже переданной границы
# возвращает датайреймы общей оценки и оценок выше и ниже границы
# возвращает датафрейм вида:
#  - 'accuracy' - точность
#  - 'labels' - номер сюжета
#  - 'claster_len' - количества элементов внутри список сюжетов

# lemit = 0.6 - граница  
# title='' - дополнительный текст
# df таблица с результатами классификации
# col_plot = 'labels' - id колонки с номером сюжета
# class_predict_col = 'class_predict' - id колонки с предсказанием
# class_true_col = 'connectedness' - id колонки с реальным значением
def get_score_DF(df_classif, 
                          lemit = 0.6, title='',
                          col_plot = 'labels', 
                          class_predict_col = 'class_predict', 
                          class_true_col = 'connectedness'): 
    if len(title) > 0:
        print(title)
    df = score_DF(df_classif, col_plot = col_plot, class_predict_col = class_predict_col, class_true_col =class_true_col)
    score_list = df["accuracy"].tolist()
    small_score_list = df[df['accuracy'] < lemit]["accuracy"].tolist()
    print(f'Всего класстеров {len(score_list)}. Score меньше {lemit} - у {len(small_score_list)}')
    return df, df[df['accuracy'] < lemit].copy(), df[df['accuracy'] >= lemit].copy()

## Классификация и оценка

### По изображениям

In [33]:
df_DBSCAN_clear__classif_rezult =  get_Classter_LR_predict_full(df_DBSCAN_clear)

In [34]:
# Из результатов убираем "мусорный кластер" (-1)
df_DBSCAN_clear__classif_rezult_pure = df_DBSCAN_clear__classif_rezult[df_DBSCAN_clear__classif_rezult['labels'] != -1].copy()

df_DBSCAN_clear__classif_score, df_DBSCAN_clear__classif_score_bad, df_DBSCAN_clear__classif_score_good = get_score_DF(df_DBSCAN_clear__classif_rezult_pure)

Всего класстеров 94. Score меньше 0.6 - у 42


In [35]:
temp_df = get_beans_df(df_DBSCAN_clear__classif_score)
temp_df

,accuracy,count_labels,labels,median_len
0,0.0-0.2,12,"1,14,21,22,32,34,58,61,63,69,70,80",2.0
1,0.2-0.4,7,"8,15,16,18,35,44,65",4.0
2,0.4-0.60,23,"0,3,4,6,11,12,13,23,24,25,26,27,29,36,38,48,51...",5.0
3,0.6-0.8,9,"2,7,9,10,19,20,31,43,74",25.0
4,0.8-1.0,43,"5,17,28,30,33,37,39,40,41,42,45,46,47,49,50,52...",2.0


In [36]:
df_KMeans_clear_best_classif_rezult = get_Classter_LR_predict_full(df_KMeans_clear_best) 

In [37]:
df_KMeans_clear_best__classif_score, df_KMeans_clear_best__classif_score_bad, df_KMeans_clear_best__classif_score_good = get_score_DF(df_KMeans_clear_best_classif_rezult)

Всего класстеров 93. Score меньше 0.6 - у 46


In [38]:
temp_df = get_beans_df(df_KMeans_clear_best__classif_score)
temp_df

,accuracy,count_labels,labels,median_len
0,0.0-0.2,10,"6,13,28,32,57,60,67,69,80,86",11.0
1,0.2-0.4,17,"9,17,24,25,34,46,51,54,55,58,62,65,71,79,83,84,91",14.0
2,0.4-0.60,19,"12,14,16,18,30,39,40,42,44,50,52,53,56,64,73,7...",12.0
3,0.6-0.8,17,"0,5,7,11,15,19,23,27,31,38,41,43,63,70,78,85,87",17.0
4,0.8-1.0,30,"1,2,3,4,8,10,20,21,22,26,29,33,35,36,37,45,47,...",14.0


In [39]:
save2csv(df_KMeans_clear_best_classif_rezult, param='classif_img_out/KMeans_clear_best_classif')
save2csv(df_DBSCAN_clear__classif_rezult, param='classif_img_out/DBSCAN_clear__classif')

NameError: name 'randint' is not defined